## Importando as bibliotecas

In [1]:
import pandas as pd
import numpy as np
import requests

## Importando as funções que serão utilizadas no decorrer do notebook.

In [2]:
from nba_api.stats.endpoints import leaguedashteamstats
from nba_api.stats.endpoints import commonplayoffseries
from nba_api.stats.static import teams
from nba_api.stats.endpoints import leaguedashteamshotlocations
from nba_api.stats.endpoints import leaguedashplayerstats

--------------------------------------------------------------------------------

# Organizando os parâmetros que serão utilizados.


---

## 1. Os "Four Factors" (Eficiência)
*Os quatro pilares estatísticos que correlacionam mais forte com vitórias.*

* **eFG% (Effective Field Goal Percentage)**
    * *O que mede:* Eficiência de arremesso ajustada, onde bolas de 3 valem 50% a mais.
    * *Fórmula:* $$eFG\% = \frac{FGM + 0.5 \times 3PM}{FGA}$$

* **TOV% (Turnover Percentage)**
    * *O que mede:* Desperdício de bola a cada 100 posses.
    * *Fórmula:* $$TOV\% = \frac{TOV}{Possessions} \times 100$$

* **OREB% (Offensive Rebound Percentage)**
    * *O que mede:* Porcentagem de rebotes ofensivos capturados das oportunidades disponíveis.
    * *Fórmula:* $$OREB\% = \frac{OREB}{OREB + Opp\_DREB}$$

* **FTR (Free Throw Rate)**
    * *O que mede:* Capacidade de gerar pontos fáceis na linha de lance livre.
    * *Fórmula:* $$FTR = \frac{FTA}{FGA}$$

---

## 2. Estilo de Jogo e Identidade
*Métricas que definem "como" o time joga, independente de ganhar ou perder.*

* **Pace (Ritmo)**
    * *Definição:* Estimativa do número de posses de bola por 48 minutos.
    * *Interpretação:* Alto (>100) = Jogo rápido/transição. Baixo (<96) = Jogo de meia-quadra/cadenciado.

* **3PAr (3-Point Attempt Rate)**
    * *O que mede:* Preferência pelo arremesso de longa distância.
    * *Fórmula:* $$3PAr = \frac{3PA}{FGA}$$

* **Rim Frequency (% FGA at Rim)**
    * *O que mede:* Agressividade em atacar a cesta (arremessos dentro de 0-4 pés).
    * *Fórmula:* $$\frac{FGA_{Rim}}{FGA_{Total}}$$

---

## 3. Desempenho Geral (Ratings)
*Métricas normalizadas por 100 posses de bola.*

* **Offensive Rating (Ortg):** Pontos marcados por 100 posses.
* **Defensive Rating (Drtg):** Pontos sofridos por 100 posses.
* **Net Rating:** $$Ortg - Drtg$$ (Diferencial de pontos por 100 posses).

---

## 4. Coletividade e Caos Defensivo
*Playmaking e criação de eventos defensivos.*

* **AST% (Assist Percentage)**
    * *O que mede:* Porcentagem de cestas de quadra que foram assistidas. Indica circulação de bola vs. *Hero Ball*.

* **STL% (Steal Percentage)**
    * *O que mede:* Porcentagem de posses adversárias que terminam em roubo de bola.

* **BLK% (Block Percentage)**
    * *O que mede:* Porcentagem de arremessos de 2 pontos do adversário que foram bloqueados.

* **DREB% (Defensive Rebound Percentage)**
    * *O que mede:* Capacidade de encerrar a posse defensiva garantindo o rebote.

---

## 5. Construção de Elenco (Roster Metrics)
*Métricas focadas na distribuição de responsabilidade.*

* **Heliocentrism (Top 2 Usage Avg)**
    * *Definição:* A média da `USG%` dos dois jogadores com maior uso no time.
    * *Objetivo:* Identificar dependência excessiva de superestrelas.

* **Bench Points %**
    * *Definição:* Porcentagem dos pontos totais do time marcados por reservas.
    * *Objetivo:* Medir profundidade do elenco.

-------------------------------------------------------------------------------------------------------------------------------------------------------------

# Fazendo o Dataframe inicial dos ultimos 10 campeões
### Primeiramente colocarei, além da temporada, o 'full_name' do time, a temporada e o ID de cada time.

In [3]:
seasons = ["2015-16","2016-17","2017-18","2018-19","2019-20","2020-21",
           "2021-22","2022-23","2023-24","2024-25"]

teams_Series = ["CLEVELAND CAVALIERS", "GOLDEN STATE WARRIORS",
                      "GOLDEN STATE WARRIORS", "TORONTO RAPTORS","LOS ANGELES LAKERS",
                      "MILWAUKEE BUCKS", "GOLDEN STATE WARRIORS", "BOSTON CELTICS",
                        "DENVER NUGGETS", "OKLAHOMA CITY THUNDER"]

indexs = ["2016_CLE", "2017_GSW", "2018_GSW", "2019_TOR", "2020_LAL", "2021_MIL", "2022_GSW", "2023_BOS", "2024_DEN", "2025_OKC"]

df_last_10_champions = pd.DataFrame(index=indexs)
df_last_10_champions["TEAMS"] = teams_Series
df_last_10_champions["SEASON"] = seasons
df_last_10_champions

,TEAMS,SEASON
2016_CLE,CLEVELAND CAVALIERS,2015-16
2017_GSW,GOLDEN STATE WARRIORS,2016-17
2018_GSW,GOLDEN STATE WARRIORS,2017-18
2019_TOR,TORONTO RAPTORS,2018-19
2020_LAL,LOS ANGELES LAKERS,2019-20
2021_MIL,MILWAUKEE BUCKS,2020-21
2022_GSW,GOLDEN STATE WARRIORS,2021-22
2023_BOS,BOSTON CELTICS,2022-23
2024_DEN,DENVER NUGGETS,2023-24
2025_OKC,OKLAHOMA CITY THUNDER,2024-25


In [ ]:
def find_ID_by_name(team_name):
    team_infos = teams.find_teams_by_full_name(team_name)
    return team_infos[0]["id"]

def find_abb_by_name(team_name):
    team_infos = teams.find_teams_by_full_name(team_name)
    return team_infos[0]["abbreviation"]

def getting_ID_row_by_name(teams_Series, df_teams):
    dict_times_ids = {}
    for time in teams_Series:
        dict_times_ids[time] = find_ID_by_name(time)

    for time in df_teams["TEAMS"]:
        for k in dict_times_ids.keys():
            if time == k:
                df_teams.loc[df_teams["TEAMS"] == time, "TEAM_ID"] = str(dict_times_ids[time])

# Será feito os datasets das métricas da liga em cada temporada

In [16]:
nba_teams = [
    # Conferência Leste
    "ATLANTA HAWKS", "BOSTON CELTICS", "BROOKLYN NETS", "CHARLOTTE HORNETS", 
    "CHICAGO BULLS", "CLEVELAND CAVALIERS", "DETROIT PISTONS", "INDIANA PACERS", 
    "MIAMI HEAT", "MILWAUKEE BUCKS", "NEW YORK KNICKS", "ORLANDO MAGIC", 
    "PHILADELPHIA 76ERS", "TORONTO RAPTORS", "WASHINGTON WIZARDS",
    
    # Conferência Oeste
    "DALLAS MAVERICKS", "DENVER NUGGETS", "GOLDEN STATE WARRIORS", "HOUSTON ROCKETS", 
    "LOS ANGELES CLIPPERS", "LOS ANGELES LAKERS", "MEMPHIS GRIZZLIES", 
    "MINNESOTA TIMBERWOLVES", "NEW ORLEANS PELICANS", "OKLAHOMA CITY THUNDER", 
    "PHOENIX SUNS", "PORTLAND TRAIL BLAZERS", "SACRAMENTO KINGS", 
    "SAN ANTONIO SPURS", "UTAH JAZZ"
]

df_teams = pd.DataFrame(index = range(0,30))
df_teams["TEAMS"] = nba_teams

getting_ID_row_by_name(df_teams["TEAMS"], df_teams)
df_teams


,TEAMS,TEAM_ID
0,ATLANTA HAWKS,1610612737
1,BOSTON CELTICS,1610612738
2,BROOKLYN NETS,1610612751
3,CHARLOTTE HORNETS,1610612766
4,CHICAGO BULLS,1610612741
5,CLEVELAND CAVALIERS,1610612739
6,DETROIT PISTONS,1610612765
7,INDIANA PACERS,1610612754
8,MIAMI HEAT,1610612748
9,MILWAUKEE BUCKS,1610612749


----------------------------------------------------------------------------------------------------------------------

## Para fazer de cada season:

In [7]:
indexs = []
for x in nba_teams:
    indexs.append(find_abb_by_name(x))
    
indexs_2016 = ['2016_' + item for item in indexs]
indexs_2017 = ['2017_' + item for item in indexs]
indexs_2018 = ['2018_' + item for item in indexs]
indexs_2019 = ['2019_' + item for item in indexs]
indexs_2020 = ['2020_' + item for item in indexs]
indexs_2021 = ['2021_' + item for item in indexs]
indexs_2022 = ['2022_' + item for item in indexs]
indexs_2023 = ['2023_' + item for item in indexs]
indexs_2024 = ['2024_' + item for item in indexs]
indexs_2025 = ['2025_' + item for item in indexs]

### Buscando as métricas

In [8]:
# Obtendo EFG_PCT, FT_RATE, ORB% e TOV%
def get_four_factors(season):
    four_factors = leaguedashteamstats.LeagueDashTeamStats(
    season=season, 
    measure_type_detailed_defense='Four Factors').get_data_frames()[0]
    cols_interesse = ["TEAM_ID", "EFG_PCT", "FTA_RATE", "OREB_PCT", "TM_TOV_PCT"]
    return four_factors[cols_interesse]

In [9]:
# Obtendo OFF_RATING, DEF_RATING, NET_RATING, AST% E PACE.
def get_advanced_stats(season):
    adv_stats = leaguedashteamstats.LeagueDashTeamStats(
    season=season, 
    measure_type_detailed_defense='Advanced').get_data_frames()[0]
    cols_interesse = ['TEAM_ID', 'OFF_RATING', 'DEF_RATING', 'NET_RATING', 'AST_PCT', 'PACE']
    return adv_stats[cols_interesse]


In [10]:
# Obtendo 3PT_RATE
def get_scoring_stats(season):
    scoring_stats = leaguedashteamstats.LeagueDashTeamStats(
    season=season, 
    measure_type_detailed_defense='Scoring').get_data_frames()[0]
    cols_interesse = ["TEAM_ID", 'PCT_FGA_3PT']
    return scoring_stats[cols_interesse]

In [11]:
# Obtendo BENCH_PTS%
def get_bench_point_percent(season):
    bench = leaguedashteamstats.LeagueDashTeamStats(
        season=season, 
        measure_type_detailed_defense='Base',
        starter_bench_nullable='Bench').get_data_frames()[0]
    
    team = leaguedashteamstats.LeagueDashTeamStats(
        season=season, 
        measure_type_detailed_defense='Base',
    ).get_data_frames()[0]

    cols_bench = ["TEAM_ID", 'PTS']
    cols_team = ["TEAM_ID", 'PTS']
    bench = bench[cols_bench]
    bench = bench.rename(columns={'PTS':'PTS_BENCH'})
    team = team[cols_team]
    df_total = pd.merge(bench, team, on="TEAM_ID")
    df_total["BENCH_PTS_PCT"] = (df_total["PTS_BENCH"] / df_total["PTS"])
    df_total = df_total.drop(columns={"PTS_BENCH", "PTS"})
    return df_total

In [12]:
# Obtendo STEAL% E BLK%
def get_shot_locations_pct(season):
    shot_locs = leaguedashteamshotlocations.LeagueDashTeamShotLocations(
        season=season,
        measure_type_simple='Base'
    ).get_data_frames()[0]
    all_shots =  leaguedashteamstats.LeagueDashTeamStats(
        season=season, 
        measure_type_detailed_defense='Base',
    ).get_data_frames()[0]

    # Fazendo as colunas de interesse (nessa função, elas estão em tuplas)
    col_ra= [('Restricted Area', 'FGA')]
    col_mid_range = [('Mid-Range', 'FGA')]
    col_id = [('', 'TEAM_ID')]

    # Inserindo as colunas no nosso DataFrame
    df_total = pd.DataFrame()
    df_total["MIDRANGE_FGA"] = shot_locs[col_mid_range]
    df_total["RIM_FGA"] = shot_locs[col_ra]
    df_total["FGA"] = all_shots["FGA"]

    # Calculando o RIM% e MID-RANGE%
    df_final = pd.DataFrame()
    df_final["TEAM_ID"] = shot_locs[col_id]
    df_final["RIM_PCT"] = df_total["RIM_FGA"] / df_total['FGA']
    df_final["MID_PCT"] = df_total["MIDRANGE_FGA"] / df_total['FGA']

    return df_final

In [13]:
def get_block_stl_pct(season):
    opponent = leaguedashteamstats.LeagueDashTeamStats(
        season=season,
        measure_type_detailed_defense='Opponent'
    ).get_data_frames()[0]

    team = leaguedashteamstats.LeagueDashTeamStats(
        season=season,
        measure_type_detailed_defense='Base'
    ).get_data_frames()[0]

    df_completo = pd.DataFrame()
    df_completo['OPP_FGA'] = opponent["OPP_FGA"]
    df_completo['BLK'], df_completo['STL'],  = team["BLK"], team['STL']
    
    df_final = pd.DataFrame()
    df_final["TEAM_ID"] = opponent["TEAM_ID"]
    df_final["STL_PCT"] = df_completo['STL'] / df_completo['OPP_FGA']
    df_final["BLK_PCT"] = df_completo['BLK'] / df_completo['OPP_FGA']

    return df_final

In [ ]:
def get_usage_stars(season):

    # Estabelecendo o número mínimo de jogos para a 'estrela'
    min_games=30

    df = leaguedashplayerstats.LeagueDashPlayerStats(
        season=season,
        per_mode_detailed='PerGame',
        measure_type_detailed_defense='Advanced' 
    ).get_data_frames()[0]

    cols = ["PLAYER_NAME", "TEAM_ID", "TEAM_ABBREVIATION", "GP", "USG_PCT"]
    metric = "USG_PCT"

    df = df[cols]
    df_filtered = df[df['GP'] >= min_games].copy()
    team_ids = df_teams["TEAM_ID"].astype(int)

    # Criando listas que servirão de colunas para o df final
    leader1_name = []
    leader2_name = []
    leader1_usg = []
    leader2_usg = []
    

    for team_id in team_ids:
        team_df = df_filtered[df_filtered['TEAM_ID'] == team_id]
        
        # Cria um DataFrame ordenado pelos 2 jogadores com mais USG%
        top_2 = team_df.sort_values(by=metric, ascending=False).head(2)
        
        players_name = []
        players_usg = []
        for index, player in top_2.iterrows(): # Index iterando as linhas
            usg_val = player[metric]
            players_name.append(player['PLAYER_NAME'])
            players_usg.append(round(usg_val, 1))

        leader1_usg.append(players_usg[0])
        leader2_usg.append(players_usg[1])
        leader1_name.append(players_name[0])
        leader2_name.append(players_name[1])
        
    df_results = pd.DataFrame()
    df_results["TEAM_NAME"] = nba_teams
    df_results["LEADER 1"], df_results["USG_L1"]= leader1_name, leader2_name
    df_results["LEADER 2"], df_results["USG_L2"] = leader1_usg, leader2_usg
    return df_results